In [ ]:
import pandas as pd
import numpy as np

import os
# CHANGE THIS DIRECTORY
#os.chdir('/Users/una.lee/The Wonderful Company/Wonderful Education Data - DataTeam/Scripts/')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import winter 2021 NWEA scores
#nwea_winter = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\NWEAWinter.csv")
# import fall 2021 NWEA scores
nwea_fall = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Tests\AssessmentResults.csv")

dibs_boy = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Tests\StudentSummary.csv")

#dibs_moy = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\MOYDIBELS.csv")

In [ ]:
# import student grades from Fall and Spring - CURRENT YEAR GRADES
#LIST STU GBU GBK GBR STU.SC STU.ID STU.NM STU.GR GBK.PD GBK.NM GBR.CD GBU.CMK GBU.TG GBU.CSC
grades = pd.read_csv(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\ES\Current Grades.csv")

# import student roster
#LIST STU ENR STU.SC STU.RS STU.ID STU.NM STU.FN STU.LN STU.LF STU.GP ENR.YR ENR.SC ENR.ED ENR.LD ENR.GR ENR.ED
roster = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Roster.xlsx")
roster = roster[(roster['Leave Date'].isnull()) & (roster['Year']==2024)].copy()
roster = roster.drop_duplicates()

# import list of SpED and teachers 
#LIST STU LN FN ID GR U1 U1? CSE.DI CAR CAR.PD CAR.TN TCH.TE CAR.DS CAR.DE IF CAR.PD = 0 from Patty **USE THIS ONE**
#LIST STU TCH IDN CSE STU.ID STU.NM STU.GR TCH.TE CSE.DI CSE.ED CSE.XD IF CSE.DI > 0 AND CSE.XD = NULL
sped = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\SPED.xlsx")
sped1 = sped.sort_values(['Student ID','Date Started']).copy()
sped2 = sped.drop_duplicates(subset=['Student ID'], keep='first')
sped2.loc[sped2[(sped2['Disability']>0) | (sped2['User1'].isin(['B','O','R','S']))].index, 'Student is SpED?'] = 'Student has an IEP'
sped2.loc[sped2[(sped2['User1']=='5')].index, 'Student is SpED?'] = 'Student has 504 Plan'

# import list of Teachers for students
#LIST STU ID LN FN GR SEC MST.RM MST TCH.TE CRS CRS.CO MST.PD
#teachers = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\ILP Spring\StudentTeachers.xlsx")

#LIST STU GBU GBS GBA GBK GBR TCH STU.ID STU.NM STU.GR STU.SC GBK.GN GBR.CD GBK.NM GBK.PD TCH.TE GBU.D1 GBU.CSC GBU.CMK GBA.AN GBA.AD GBA.DD GBA.GC GBS.MK GBA.MX IF GBS.MK = " " AND GBS.MX > 0 AND GBA.GC = 1 AND GBA.DD >= GBR.SD AND GBA.DD <= GBR.ED AND GBK.SC = TCH.SC
#missing_assignments = pd.read_excel(r"C:\Users\derek.castleman\Desktop\MonthlyDataPull\February2023\MissingAssignments.xlsx")

att = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Attendance.xlsx")

#LIST AHS STU STU.SC STU.ID STU.GR AHS.YR AHS.TD AHS.TRU IF AHS.YR = 2022-2023
truant = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\2024-2025\Fall\Truant.xlsx")


In [ ]:
# import historical grades
#LIST STU HIS CRS STU.NM STU.GR STU.SC STU.ID HIS.SN HIS.CN HIS.MK HIS.GR HIS.TE HIS.YR HIS.CO HIS.CC his.rt CRS.CN CRS.CO CRS.E1
#hg_grades = pd.read_excel(r"C:\Users\derek.castleman\Desktop\ILP\HitoricalGrades.xlsx")


In [ ]:
grades

Check for any courses that are missing grades and see how they might be misnamed.

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Grades Check", filename = "Grades Check"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(grades)

In [ ]:
#roster = roster.loc[roster['School'] == 4]
#roster

In [ ]:
#start_date.to_csv('~/Documents/WCPA/Progress Reports/Output/District Start Date by ID 02222022.csv', index=False)


### SpED and teacher information

In [ ]:
#Add in Sped and teacher information
roster1 = roster.merge(sped2[['Student ID',
                             'Student is SpED?',
                             'Teacher name']],
                       on='Student ID', how='left').reset_index()
# Add in District Entry Date
roster2 = roster1
roster2.groupby('School')['Student ID'].count()


In [ ]:
roster2

In [ ]:
roster2 = roster2.drop(columns=['index', 'Res Schl', 'School1', 'Enter Date', 'Leave Date', 'Enter Date1'])
roster2

In [ ]:
#roster2 = roster2.loc[roster2['School'] == 4]
#roster2

### NWEA

In [ ]:
# clean up NWEA - categorize scores based on percentiles
def clean_nwea(nwea_df, growth_col, term):
    nwea_df1 = nwea_df[(nwea_df['GrowthMeasureYN']==True)&(nwea_fall['Course'].isin(['Reading','Math K-12']))].copy()

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'] <= 20].index,'NWEA Category'] = 'Significantly Below Grade Level'

    nwea_df1.loc[nwea_df1[(nwea_df1['TestPercentile'] > 20) &
             (nwea_df1['TestPercentile'] <= 40)].index,'NWEA Category'] = 'Below Grade Level'

    nwea_df1.loc[nwea_df1[(nwea_df1['TestPercentile'] > 40) &
             (nwea_df1['TestPercentile'] <= 60)].index,'NWEA Category'] = 'At Grade Level'

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'] > 60].index,'NWEA Category'] = 'Above Grade Level'

    nwea_df1.loc[nwea_df1[nwea_df1['TestPercentile'].isnull()].index,'NWEA Category'] = 'No Percentile Assigned'
    
    nwea_final = nwea_df1.pivot(index=['StudentID'], columns='Subject',values=['NWEA Category',growth_col]).reset_index()

    nwea_final.columns = ['StudentID',f"NWEA {term} Language Arts",
                      f"NWEA {term} Mathematics",
                      f"NWEA {term} Growth Goal Met - Language Arts",
                      f"NWEA {term} Growth Goal Met - Mathematics" ]
    return nwea_final

#nwea_winter_clean = clean_nwea(nwea_winter,'FallToWinterMetProjectedGrowth','Winter 2021')
nwea_fall_clean = clean_nwea(nwea_fall,'FallToFallMetProjectedGrowth','Fall 2023')
#nwea_final = nwea_winter_clean.merge(nwea_fall_clean[['StudentID',
                                                      #'NWEA Fall 2021 Language Arts',
                                                      #'NWEA Fall 2021 Mathematics']], on='StudentID', how='outer')

In [ ]:
nwea_fall = nwea_fall_clean

In [ ]:
nwea_fall.info()

In [ ]:
nwea_fall[:2]


In [ ]:
nwea_final = nwea_fall
nwea_final

In [ ]:
roster2 = pd.merge(roster2, nwea_final, how='left', left_on='Student ID', right_on='StudentID')
roster2

### Dibels

In [ ]:
dibs = dibs_boy
dibs

In [ ]:
# Reformat Dibels data
#Keep highest score from BOY or MOY - keep last
dibs_sorted = dibs.sort_values(['Primary ID - Student ID (District ID)',
                                'Benchmark Period',
                                'Composite Score'])

#Drop duplicate scores - keep highest
dibs2 = dibs_sorted.drop_duplicates(subset=['Primary ID - Student ID (District ID)',
                                            'Benchmark Period'],keep='last')
#Check for duplicates
dibs2[dibs2.duplicated(subset=['Primary ID - Student ID (District ID)',
                               'Benchmark Period'],
                       keep=False)][['Primary ID - Student ID (District ID)',
                                     'Benchmark Period',
                                     'Composite Level']]
#Pivot scores
#dibs_final = dibs2.pivot(index='Primary ID - Student ID (District ID)',
            #columns='Benchmark Period',
            #values='Composite Level').reset_index()
#dibs_final.rename(columns={'BOY':'Dibels BOY Benchmark',
                          #'MOY':'Dibels MOY Benchmark',
                          #'Primary ID - Student ID (District ID)':'Student ID'}, 
        #inplace=True)
dibs_final = dibs2[['Primary ID - Student ID (District ID)', 'Composite Level']]
dibs_final = dibs_final.fillna('No Score')


In [ ]:
dibs_boy = dibs_final
dibs_boy

In [ ]:
dibs_boy = dibs_boy.rename(columns = {'Primary ID - Student ID (District ID)':'Student ID', 'Composite Level': 'DIBELS Level'})
dibs_boy

In [ ]:
fix = pd.read_excel(r"C:\Users\derek.castleman\Desktop\PrintQueryToExcel_20241010_101244_6411795.xlsx")
fix

In [ ]:
fix = pd.merge(dibs_boy, fix, how='inner', left_on='Primary ID - Student ID (District ID)', right_on='State Student ID')
fix

In [ ]:
dibs_boy = fix[['Student ID', 'Composite Level']]
dibs_boy

In [ ]:
roster2 = pd.merge(roster2, dibs_boy, how='left', left_on='Student ID', right_on='Student ID')
roster2

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Delano ES Assessments", filename = "Delano ES Assessments"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(roster2)

### Attendance

In [ ]:
att

In [ ]:
att['Q1 Attendance'] = att['% Present'] *100
att = att.round({'Q1 Attendance': 0})
att

In [ ]:
final_att = att[['Student ID', 'Q1 Attendance', 'Unexcused Absences']]
final_att

In [ ]:
att = final_att
att

### Missing Assignments

In [ ]:
#ma  = missing_assignments

#ma


In [ ]:
#ma['Missing Assignments'] = 1
#ma

In [ ]:
#ma2 = ma.groupby(by=['Student ID'])['Missing Assignments'].sum().reset_index()
#ma2

## Tardies and Truancies

In [ ]:
truant

In [ ]:
truant = truant.groupby(by=['Student ID'])['Tardies', 'Days of Truancy'].sum().reset_index()
truant

In [ ]:
att = pd.merge(att, truant, on='Student ID', how='outer')
att

In [ ]:
roster2 = pd.merge(roster2, att, how='left', on='Student ID')
roster2

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Delano ES Attendance", filename = "Delano ES Attendance"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(roster2)

### Current grades

### Current grades

In [ ]:
grades


In [ ]:
grades.info()


In [ ]:
grades

In [ ]:
grades['Course Number'] = grades.groupby(['Student ID']).cumcount()+1

grades_courses = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Course title').reset_index()
grades_grades = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Mark1').reset_index()
grades_teachers = grades.pivot(index=['Student ID'],
                                      columns='Course Number',
                                      values='Teacher name').reset_index()

grades_courses_name = {1:'Quarter 1 Course 1',
                 2:'Quarter 1 Course 2',
                 3:'Quarter 1 Course 3',
                 4:'Quarter 1 Course 4',
                 5:'Quarter 1 Course 5',
                 6:'Quarter 1 Course 6',
                 7:'Quarter 1 Course 7',
                 8:'Quarter 1 Course 8',
                 9:'Quarter 1 Course 9',
                     10:'Quarter 1 Course 10'}

grades_grades_name = {1:'Quarter 1 Course 1 Grade',
                 2:'Quarter 1 Course 2 Grade',
                 3:'Quarter 1 Course 3 Grade',
                 4:'Quarter 1 Course 4 Grade',
                 5:'Quarter 1 Course 5 Grade',
                 6:'Quarter 1 Course 6 Grade',
                 7:'Quarter 1 Course 7 Grade',
                 8:'Quarter 1 Course 8 Grade',
                 9:'Quarter 1 Course 9 Grade',
                    10:'Quarter 1 Course 10 Grade'}

grades_teachers_name = {1:'Quarter 1 Course 1 Teacher',
                 2:'Quarter 1 Course 2 Teacher',
                 3:'Quarter 1 Course 3 Teacher',
                 4:'Quarter 1 Course 4 Teacher',
                 5:'Quarter 1 Course 5 Teacher',
                 6:'Quarter 1 Course 6 Teacher',
                 7:'Quarter 1 Course 7 Teacher',
                 8:'Quarter 1 Course 8 Teacher',
                 9:'Quarter 1 Course 9 Teacher',
                      10:'Quarter 1 Course 10 Teacher'}

grades_courses.rename(columns=grades_courses_name, inplace=True)
grades_grades.rename(columns=grades_grades_name, inplace=True)
grades_teachers.rename(columns=grades_teachers_name, inplace=True)

In [ ]:
grades_courses

In [ ]:
grades_grades

In [ ]:
grades_teachers

In [ ]:
merge_one = pd.merge(grades_courses, grades_grades, how = 'outer', on = 'Student ID')
merge_one

In [ ]:
grades_final = pd.merge(merge_one, grades_teachers, how = 'outer', on = 'Student ID')
grades_final

In [ ]:
roster2

In [ ]:
final = pd.merge(roster2, grades_final, how='left', on='Student ID')
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Delano ES Redone ILP", filename = "Delano ES Redone ILP"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)